**1. WEBSCRAPPING**

In [30]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

# Fonction pour récupérer les commentaires
def get_comment(commentaires, i):
    try:
        return commentaires[i].text
    except IndexError:
        return "No comment"

# Fonction pour extraire les avis d'une page
def extract_reviews(html, pageNumber):
    soup = BeautifulSoup(html, 'html.parser')

    noms = soup.select('span[data-consumer-name-typography]')
    notes = soup.select('div[data-service-review-rating]')
    domiciles = soup.select('div[data-consumer-country-typography] span')
    nb_avis = soup.select('span[data-consumer-reviews-count-typography]')
    date_posts = soup.select('time[data-service-review-date-time-ago]')
    date_experiences = soup.select('p[data-service-review-date-of-experience-typography]')
    commentaires = soup.select('div[data-review-content] p[data-service-review-text-typography]')
    titres = soup.select('h2[data-service-review-title-typography]')

    reviews = []

    for i in range(len(noms)):
        reviews.append({
            "nom": noms[i].string or "",
            "note": notes[i]['data-service-review-rating'] or "",
            "domicile": domiciles[i].string or "",
            "nb_avi": nb_avis[i].text or "",
            "date_post": date_posts[i].text or "",
            "date_experience": date_experiences[i].text or "",
            "commentaire": get_comment(commentaires, i),
            "titre": titres[i].text or ""
        })

    return reviews

# Fonction pour scraper une page
def fetch_url(pageNumber):
    webpage = requests.get(fr"https://www.trustpilot.com/review/www.cdiscount.com?languages=fr&page={pageNumber}")
    return extract_reviews(webpage.content, pageNumber)

# Définir les plages de pages à scraper
start_page = 6251
end_page = 6485
cdiscount_reviews_page_numbers = range(start_page, end_page + 1)

# Ouvrir le fichier CSV en mode 'a' pour ajouter les données
with open("1. Résultats Wepscrapping.csv", "a", newline="", encoding="utf-8") as file:
    # Parcourir les pages et ajouter les avis au fichier CSV au fur et à mesure
    for page_number in cdiscount_reviews_page_numbers:
        reviews = fetch_url(page_number)
        df = pd.DataFrame(reviews)
        df.to_csv(file, index=False, header=not file.tell())  # Ne pas écrire l'en-tête si le fichier n'est pas vide